# 🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB

Objetivo: dominar la conexión, modelado básico, consultas eficientes e inserciones masivas en PostgreSQL (relacional) y MongoDB (NoSQL), con prácticas de índices, agregaciones y particionado.

- Duración: 90-120 min
- Dificultad: Media
- Prerrequisitos: Python y SQL básicos, nociones de modelado de datos

## Requisitos y entorno

- Este notebook incluye bloques opcionales para conectarte a PostgreSQL y MongoDB reales si tienes servicios disponibles.
- También incluye demostraciones auto-contenidas con SQLite para la parte SQL.
- Variables de entorno sugeridas: `POSTGRES_URI`, `MONGODB_URI`.
- Opcional: usa Docker para levantar servicios locales de Postgres y MongoDB.

## 1. SQL con SQLAlchemy (demo con SQLite)

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

# Usa PostgreSQL si POSTGRES_URI está disponible; de lo contrario, SQLite en memoria
pg_uri = os.getenv('POSTGRES_URI')
engine = create_engine(pg_uri) if pg_uri else create_engine('sqlite+pysqlite:///:memory:', echo=False, future=True)

with engine.begin() as conn:
    conn.execute(text('''
        CREATE TABLE IF NOT EXISTS ventas (
          id INTEGER PRIMARY KEY,
          fecha TEXT,
          cliente_id INTEGER,
          producto_id INTEGER,
          cantidad INTEGER,
          precio REAL,
          total REAL
        );
    '''))

df = pd.DataFrame({
    'id': [1,2,3,4],
    'fecha': ['2025-10-01','2025-10-02','2025-10-02','2025-10-03'],
    'cliente_id': [10,10,11,12],
    'producto_id': [101,102,101,103],
    'cantidad': [1,2,1,3],
    'precio': [100.0, 50.0, 100.0, 20.0]
})
df['total'] = df['cantidad'] * df['precio']
df.to_sql('ventas', engine, if_exists='append', index=False)

with engine.connect() as conn:
    res = conn.execute(text('SELECT cliente_id, SUM(total) as total FROM ventas GROUP BY cliente_id ORDER BY total DESC'))
    rows = res.fetchall()
rows

### 1.1 Índices y optimización (PostgreSQL)

In [ ]:
# Estas sentencias funcionan si usas PostgreSQL real
sql_index = '''
CREATE INDEX IF NOT EXISTS idx_ventas_cliente_fecha ON ventas (cliente_id, fecha);
ANALYZE ventas;
EXPLAIN ANALYZE SELECT * FROM ventas WHERE cliente_id = 10 AND fecha >= '2025-10-02';
'''
print(sql_index)

## 2. MongoDB con PyMongo

In [ ]:
from datetime import datetime
try:
    import pymongo
    MONGODB_URI = os.getenv('MONGODB_URI', 'mongodb://localhost:27017')
    # Descomenta para usar una base real
    # client = pymongo.MongoClient(MONGODB_URI)
    # db = client['demo']
    # col = db['ventas']
    # col.insert_many([
    #     {'_id':1,'fecha': datetime(2025,10,1), 'cliente_id':10, 'total':100.0},
    #     {'_id':2,'fecha': datetime(2025,10,2), 'cliente_id':10, 'total':100.0},
    #     {'_id':3,'fecha': datetime(2025,10,2), 'cliente_id':11, 'total':100.0},
    # ])
    # col.create_index([('cliente_id', 1), ('fecha', 1)])
    # pipeline = [
    #   {'$match': {'fecha': {'$gte': datetime(2025,10,2)}}},
    #   {'$group': {'_id': '$cliente_id', 'total': {'$sum': '$total'}}},
    #   {'$sort': {'total': -1}}
    # ]
    # list(col.aggregate(pipeline))
    print('PyMongo disponible; conecta si tienes un servidor MongoDB.')
except Exception as e:
    print('PyMongo no disponible o sin conexión:', e)

## 3. Patrones de modelado y buenas prácticas

- PostgreSQL: normalización moderada, claves surrogate, índices compuestos, particionamiento por rango/fecha.
- MongoDB: diseñar según patrones de acceso; embedding vs referencing; TTL e índices parciales.
- Carga: upserts idempotentes, lotes (batch size) y manejo de transacciones cuando aplique.
- Observabilidad: logging estructurado y métricas por operación.

## 4. Ejercicios

1. Diseña un índice compuesto para acelerar consultas por `producto_id` y `fecha`.
2. Escribe una consulta de ventas acumuladas por día (window function en PostgreSQL).
3. En MongoDB, crea un pipeline para obtener el top 3 de clientes por gasto en un rango de fechas.